In [3]:


from IPython.display import HTML

HTML('''
<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>
''')


# Data Science tools on Canada's Big  Cities

In [4]:
import pandas as pd
import os

import numpy as np

from scipy.stats import chi2_contingency
from plot_post import plot_post

# Visualization libraries
import matplotlib
matplotlib.rcParams.update({'font.size': 20})
import matplotlib.pyplot as plt
#plt.rcParams['font.size'] = 20
# Seaborn for visualization
import seaborn as sns
#sns.set(font_scale = 1)
import prince # Correspondenc Analysis (CA)

%matplotlib inline  


In [5]:
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display
import ipywidgets as widgets


In [6]:
pc_gt_comp_df =  pd.read_csv('data/pc_gt_comp_df.csv', index_col = 0)
tools_df = pd.read_csv('data/tools_df.csv', index_col = 0)
tools_df.columns.rename('Tools', inplace=True)
tools_df.columns = pd.Series([c.capitalize() for c in tools_df.columns], name = 'Tools')
tools_df.index = pd.Series([c.split(',')[0].capitalize() for c in tools_df.index], name = 'Cities')


In [7]:
#  Make ccrrespondence Analysis
import seaborn as sns

ca = prince.CA(
     n_components=2,
     n_iter=3,
     copy=True,
     check_input=True,
     engine='auto',
     random_state=42
     )

ca = ca.fit(tools_df)

In [8]:
target_city = widgets.Dropdown(options = list(tools_df.index), value = 'Montreal')
measure = widgets.Dropdown(options = list(pc_gt_comp_df.columns), value = 'Certainty')
# An HBox lays out its children horizontally
ui = widgets.HBox([target_city, measure])

column_coords = ca.column_coordinates(tools_df)
row_coords = ca.row_coordinates(tools_df)

def interactive_plot(target_city, measure ):
    print(target_city)
    mask = sorted([i for i in pc_gt_comp_df.index if target_city in i] )
    city_df = pc_gt_comp_df.loc[mask,:]
    city_df.index = [i[0:i.rfind('_')].capitalize() for i in city_df.index]
    city_df
    city_coords = column_coords.join(city_df)
    city_coords
    plot_df = city_coords.reset_index().drop('index',axis = 1)

    ax = ca.plot_coordinates(
         X=tools_df,
         ax=None, 
         x_component=0,
         y_component=1,
         show_row_labels=True,
         show_col_labels=True,
         figsize = (14,10) )

    sns.scatterplot(plot_df.iloc[:,0], plot_df.iloc[:,1], s = 120, alpha = 1
                    , hue =[round(i,2) for i in plot_df[measure]]
                    , palette = sns.diverging_palette(220, 10, sep=120, n=len(column_coords.index))
                                                  )
    
    plt.legend(loc='upper left')
    plt.xlabel('Component 1  {:.2f}% (inertia)'.format((ca.explained_inertia_[0])))
    plt.ylabel('Component 1  {:.2f}% (inertia)'.format((ca.explained_inertia_[1])))
    matplotlib.rcParams["font.size"] = 16
    plt.show()


In [9]:
out = widgets.interactive_output(interactive_plot, {'target_city': target_city, 'measure': measure})
display(ui, out)

Output()

In [10]:
tools_df

Tools,Python,R,Tableau,Spark,Sql,Hadoop,Sas_others
Cities,,,,,,,
Montreal,38,15,5,27,25,25,14
Ottawa,19,16,2,9,11,3,5
Toronto,111,79,43,41,78,38,51
Vancouver,30,22,13,13,18,12,5
